In [1]:
import torch
import torch.nn as nn
import torchvision.models as models

model = models.resnet50(weights=None)
in_features = model.fc.in_features
cls_num = 3

# load pretrained model
plantleaf_pretrained = torch.load('./models/byol/resnet/plantleaf_byol_resnet50.pth')
model.load_state_dict(plantleaf_pretrained)

# fc layer weight initialization
weights_dist = {}
for name, layer in model.named_modules():
    if hasattr(layer, 'weight') and isinstance(layer.weight, torch.nn.parameter.Parameter):
        weights_dist[name] = (layer.weight.data.mean().item(), layer.weight.data.std().item())

last_layer_name = list(weights_dist.keys())[-1]
mean, std = weights_dist[last_layer_name]

model.fc = nn.Linear(in_features, cls_num)
model.fc.weight.data.normal_(mean, std)
model.fc.bias.data.zero_()

tensor([0., 0., 0.])